In [1]:
import os
import pandas as pd
import json
import glob

In [2]:
def process_folder(folder_path):
    # Extract configuration from folder name
    folder_name = os.path.basename(folder_path)
    requested_mbps, duration, packet_length, cpu_limit, _,_ = folder_name.split('_')
    
    power_consumptions = []
    cpu_usages = []
    
    # Process iperf server log
    server_file = glob.glob(os.path.join(folder_path, "log_iperf_server_*.csv"))
    if server_file:
        server_file = server_file[0]
        df_server = pd.read_csv(server_file)
        
        # Calculate the average Mbps effectively received at server using 'Transfer' column
        # Convert each interval's transfer (in bytes) to Mbps and then calculate the average
        df_server['Mbps'] = df_server['Bitrate'] / 1e6  # Convert each interval's Transfer from bps to Mbps
        avg_mbps_received = df_server['Mbps'].mean()  # Average Mbps over all intervals
        
    # Process power consumption data
    power_file = os.path.join(folder_path, "metrics_energy.json")
    if os.path.exists(power_file):
        with open(power_file, 'r') as f:
            power_data = json.load(f)
        
        # Collect power values for CU container
        for pod_data in power_data.values():
            for entry in pod_data:
                power_consumptions.append(entry['value'])
        
        # Calculate average power consumption for CU
        avg_power_consumption = sum(power_consumptions) / len(power_consumptions) if power_consumptions else 0
    else:
        avg_power_consumption = 0  # Default if no data found
    
    # Process cpu usage data
    cpu_file = os.path.join(folder_path, "metrics_cpu.json")
    if os.path.exists(cpu_file):
        with open(cpu_file, 'r') as f:
            cpu_data = json.load(f)
        
        # Collect power values for CU container
        for pod_data in cpu_data.values():
            for entry in pod_data:
                cpu_usages.append(entry['value'])
        
        # Calculate average power consumption for CU
        avg_cpu_usage = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0
    else:
        avg_cpu_usage = 0  # Default if no data found
    

    # Return row data
    return {
        "cpu limit at cu": cpu_limit,
        "Mbps requested": int(requested_mbps),
        "packet length requested": int(packet_length),
        "avg Mbps effectively received at server": avg_mbps_received,
        "avg cu power consumption [W]": avg_power_consumption,
        "avg cu cpu usage [%]": avg_cpu_usage,
    }



In [3]:
def main(base_folder_path):
    data = []
    # Iterate over each sub-folder in the base folder
    for folder in os.listdir(base_folder_path):
        folder_path = os.path.join(base_folder_path, folder)
        if os.path.isdir(folder_path):
            row = process_folder(folder_path)
            data.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Path to the base folder containing all experiment folders
base_folder_path = "/home/ubuntu/power-consumption-tool/power-metrics-per-pod-realtime/test_automation/exp1/data"
df = main(base_folder_path)

# Save DataFrame to CSV
# df.to_csv("summary_metrics.csv", index=False)
df = df[df['packet length requested'] == 700]
df = df[df['Mbps requested'] == 70]
df['cpu limit at cu'] = df['cpu limit at cu'].str.replace('mi', '').astype(int)
df = df.sort_values(by='cpu limit at cu')
df


,cpu limit at cu,Mbps requested,packet length requested,avg Mbps effectively received at server,avg cu power consumption [W],avg cu cpu usage [%]
33,50,70,700,21.265017,0.072902,76.950448
34,75,70,700,30.308456,0.079238,80.994576
29,100,70,700,39.010779,0.086131,85.085537
21,125,70,700,49.836729,0.088392,86.701604
24,150,70,700,57.881279,0.093054,78.818302
5,175,70,700,65.246931,0.094043,89.042137
23,200,70,700,71.372083,0.109423,90.350929
20,500,70,700,73.176781,0.109591,95.245857
25,1000,70,700,72.945505,0.110610,92.970456
